In [78]:
import numpy as np

In [79]:
names = ["c1", "c2", "c3", "c4", "c5", "c6", "c7", "c8"]

name_to_index_map = {}
for index, name in enumerate(names):
    name_to_index_map[name] = index

In [82]:
# https://en.wikipedia.org/wiki/3-opt

def successor(index):
    index += 1
    if(index >= len(names)):
        index = 0
    return index

def name_to_index(name):
    if(type(name) is list):
        indices = []
        for n in name:
            indices.append(name_to_index_map[n])
        return indices
    return name_to_index_map[name]

def index_to_name(index):
    if(type(index) == list):
        n = []
        for i in index:
            n.append(names[i])
        return n
    return names[index]

def distance(mat, a, b):
    return mat[a,b]

def all_segments(n: int):
    """Generate all segments combinations"""
    return ((i, j, k)
        for i in range(n)
        for j in range(i + 2, n)
        for k in range(j + 2, n + (i > 0)))

def three_opt_segment(mat, tour, i, j, k):
    """If reversing tour[i:j] would make the tour shorter, then do it."""
    # Given tour [...A-B...C-D...E-F...]
    A, B, C, D, E, F = tour[i-1], tour[i], tour[j-1], tour[j], tour[k-1], tour[k % len(tour)]
    d0 = distance(mat, A, B) + distance(mat, C, D) + distance(mat, E, F)
    d1 = distance(mat, A, C) + distance(mat, B, D) + distance(mat, E, F)
    d2 = distance(mat, A, B) + distance(mat, C, E) + distance(mat, D, F)
    d3 = distance(mat, A, D) + distance(mat, E, B) + distance(mat, C, F)
    d4 = distance(mat, F, B) + distance(mat, C, D) + distance(mat, E, A)

    if d0 > d1:
        tour[i:j] = reversed(tour[i:j])
        return -d0 + d1
    elif d0 > d2:
        tour[j:k] = reversed(tour[j:k])
        return -d0 + d2
    elif d0 > d4:
        tour[i:k] = reversed(tour[i:k])
        return -d0 + d4
    elif d0 > d3:
        tmp = tour[j:k] + tour[i:j]
        tour[i:k] = tmp
        return -d0 + d3
    return 0

def three_opt(mat, tour):
    """Iterative improvement based on 3 exchange."""
    while True:
        delta = 0
        for (a, b, c) in all_segments(len(tour)):
            delta += three_opt_segment(mat, tour, a, b, c)
        if delta >= 0:
            break
    return tour

def three_opt_non_reversing(tour, i, j, k):
    tour = tour.copy()
    tmp = np.concatenate((tour[j+1:k+1], tour[i+1:j+1]))
    tour[i+1:k+1] = tmp
    return tour

def print_3opt_input(tour, i, j, k):
    i2 = index_to_name(successor(name_to_index(i)))
    j2 = index_to_name(successor(name_to_index(j)))
    k2 = index_to_name(successor(name_to_index(k)))
    print(f"3-opt:  {i}-{i2} (1st edge)")
    print(f"3-opt:  {j}-{j2} (2nd edge)")
    print(f"3-opt:  {k}-{k2} (3rd edge)")
    print(f"input:  {'-'.join(tour)}")

def print_3opt_result(tour):
    print(f"result: {'-'.join(tour)}")

### non-reversing 3-opt

In [81]:
tour = ["c1", "c2", "c3", "c4", "c5", "c6", "c7", "c8", "c1"]
i = "c2" # first name of 1st edge 
j = "c4" # first name of 2nd edge
k = "c6" # first name of 3rd edge

# print input
print_3opt_input(tour, i, j, k)

# names to indices
tour = name_to_index(tour)
i = name_to_index(i) 
j = name_to_index(j)
k = name_to_index(k)

# compute result
tour = three_opt_non_reversing(tour, i, j, k)
tour = index_to_name(tour)

# print result
print_3opt_result(tour)

3-opt:  c2-c3 (1st edge)
3-opt:  c4-c5 (2nd edge)
3-opt:  c6-c7 (3rd edge)
input:  c1-c2-c3-c4-c5-c6-c7-c8-c1
result: c1-c2-c5-c6-c3-c4-c7-c8-c1


### reversing 3-opt

In [ ]:
# TODO
# three_opt_segment() might be buggy (wrong array indices)